In [1]:
import pandas as pd
import re
from random import randrange

Reading raw data for recipes from json files

In [2]:
path  = r'..\\recipes_raw'
file_1 = r'recipes_raw_nosource_ar.json'
file_2 = r'recipes_raw_nosource_epi.json'
file_3 = r'recipes_raw_nosource_fn.json'
file_1_df = pd.read_json(path+ r"\\"+file_1).T
file_2_df = pd.read_json(path+ r"\\"+file_2).T
file_3_df = pd.read_json(path+ r"\\"+file_3).T
appended_df = file_1_df.append(file_2_df).append(file_3_df)
appended_df = appended_df.dropna(subset = ['ingredients','instructions','title']) # Loss of 691 Rows"

In [3]:
def test_token(token):
        token_l = token.lower()
        if token_l.isdigit():
            return False
        for exp in explicit_list:
            if token == exp:
                return False
        for cont in contain_list:
            if cont in token_l:
                return False
        return True
    
def remove_by_regex(string,reg_list):
    for reg in reg_list:
        string = re.sub(reg, "",string)
    return string

def remove_by_regex_using_list(string, remove_list):
    for word in remove_list:
        dynamic_regex = "("+ word.lower() +"[s]{0,1})\W"
        string = re.sub(dynamic_regex,"",string)
    return string

def remove_by_regex_using_list_contains(string, remove_list):
    for word in remove_list:
        dynamic_regex = "("+ word.lower() +")\\w*"
        string = re.sub(dynamic_regex,"",string)
    return string

def remove_word_from_text(word, text):
    text = re.sub(word,"",text)
    return text

def if_item_contain_from_list(item,list_obj):
    for cont in list_obj:
        if cont in item:
            return True
    return False

def print_randon_ingredients(df,size):
    starting_point = randrange(0,df.shape[0] - size-1)
    file = open('Random.txt','w')
    for row in df[starting_point:(starting_point+size)].iterrows():
        for ind, it in enumerate(row[1]['ingredients']):
            file.write("Original : "+it+"\n")
            if(row[1]['filtered_ingredient'][ind]):
                file.write("Filterd_ingredient : " + str(row[1]['filtered_ingredient'][ind])+"\n")
            if(row[1]['removing_extra'][ind]):
                file.write("removing_extra     : " + str(row[1]['removing_extra'][ind])+"\n")
            file.write("\n")
    file.close()

Data Cleaning

In [4]:
%%time
regular_list = ['package','tablespoon','cup', 'ounce','can','pack','teaspoon', 'pound','halve','diced','chopped','small','shredded','all-purpose']
contain_list = ['fine','need']
regex_list = [
    "\(.*\)",
    "advertisement",
    "\\d+\\S*\\d*",
    "\S*[*]\S*"]
ban_if_contain = [
    'reynolds wrap®'
]

def filter_strings():
    dish_list = []
    for ing in appended_df.ingredients:
        ingredient_list = []
        for item_string in ing:
            item = item_string.strip().lower()
            if (not if_item_contain_from_list(item, ban_if_contain)):
                item = remove_by_regex(item, regex_list)
                item = remove_by_regex_using_list(item, regular_list)
                item = remove_by_regex_using_list_contains(item, contain_list)
                if len(item) > 0:
                    ingredient_list.append(item.strip())
        dish_list.append(ingredient_list)
        
    return dish_list

filter_ingredient_list = filter_strings()

appended_df['filtered_ing'] = pd.Series(data= filter_ingredient_list, index=appended_df.index, dtype='O')

Wall time: 24.9 s


Removal of strings using POS Tags

In [5]:
%%time
only_allowed = ['NN','NNS','JJ']
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.tokenize.treebank import TreebankWordDetokenizer
recipie_list = []
for ing in appended_df.filtered_ing:
    ingred_list = []
    for item in ing:
        for tag_p in pos_tag(word_tokenize(item)):
            if tag_p[1] not in only_allowed:
                try:
                    item = remove_word_from_text(tag_p[0],item)
                except Exception as e:
                    print(e)
                    print(tag_p)
        ingred_list.append(item)
    recipie_list.append(ingred_list)

missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
nothing to repeat at position 0
('+', 'CD')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
nothing to repeat at position 0
('?', '.')
missing ), unterminated subpattern at position 0
('(', '(')
unbalanced parenthesis at position 0
(')', ')')
missing ), unterminated s

nothing to repeat at position 0
('?', '.')
nothing to repeat at position 0
('?', '.')
missing ), unterminated subpattern at position 0
('(', '(')
unbalanced parenthesis at position 0
(')', ')')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
unbalanced parenthesis at position 0
(')', ')')
missing ), unterminated subpattern at position 0
('(', '(')
unbalanced parenthesis at position 0
(')', ')')
nothing to repeat at position 0
('?', '.')
nothing to repeat at position 0
('?', '.')
unbalanced parenthesis at position 0
(')', ')')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
missing ), unterminated subpattern at position 0
('(', '(')
unbalanced parenthesis at position 0
(')', ')')
unbalanced parenthesis at position 0
(')', ')')
missing ), unterminated subpattern at position 0
('(', '(')


In [ ]:
appended_df['Filterd_ingredient'] = pd.Series(data = recipie_list, index=appended_df.index, dtype='O')
appended_df.to_pickle('Cleaned_recipes.P')

In [87]:
# print_randon_ingredients(appended_df,100)